In [ ]:
#Outline detection code borrowed and modified from here: http://creativemorphometrics.co.vu/blog/2014/08/05/automated-outlines-with-opencv-in-python/
import os
import numpy as np
import cv2 #this is the main openCV class, the python binding file should be in /pythonXX/Lib/site-packages
from matplotlib import pyplot as plt

from scipy import ndimage
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.stats import linregress

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#The code below extracts any edges it detects in the image then only keeps the largest two outlines (presumably belonging to the channels)
scale = 2

ThreshMin = 2
ThreshMax = 255
ErosionSize = (5,5)
BlurSize = (10,10)

files = []
for filename in os.listdir('images'):
    if filename.endswith('.tif'):
        files.append(os.path.join('images', filename))

channimage = cv2.flip(cv2.bitwise_not(cv2.imread(files[10])), 0) #import and inverts image
channimageBW = cv2.cvtColor(gwash, cv2.COLOR_BGR2GRAY) #change to grayscale
channimageBW = np.array(zip(*channimageBW[::-1]))
channimageBW = cv2.blur(channimageBW,(5,5))
channimageBW = channimagehBW[50:len(channimageBW) - 1]

plt.imshow(gwashBW, 'gray', aspect='auto')
plt.show()


ret,thresh1 = cv2.threshold(channimageBW, ThreshMin, ThreshMax,cv2.THRESH_BINARY)
kernel = np.ones(ErosionSize,np.uint8) #square image kernel used for erosion
erosion = cv2.erode(thresh1, kernel,iterations = 1) #refines all edges in the binary image

opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

plt.imshow(closing, 'gray', aspect='auto') #Figure 2
plt.xticks([]), plt.yticks([])
plt.show()

test2, contours, hierarchy = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


blank = np.array([[0]*len(closing[1]) for i in range(len(closing))])
im = cv2.drawContours(blank,contours,-1,(255,255,255),1)

plt.imshow(im, 'gray', aspect='auto') #Figure 2
plt.xticks([]), plt.yticks([])
plt.show()

areas = [] #list to hold all areas

for contour in contours:
  ar = cv2.contourArea(contour)
  areas.append(ar)
    
sortareas = areas[:]
sortareas.sort(reverse = True)

max_areas_index = []

for i in range(2):
    max_areas_index.append(areas.index(sortareas[i]))
    #index of the list element with largest area

cnt = [0,0]

for i in range(2):
    cnt[i] = contours[max_areas_index[i]]

cnt = np.array(cnt)
    
blank = np.array([[0]*len(closing[1]) for i in range(len(closing))])
im = (cv2.drawContours(blank, cnt, -1, (1, 1, 1), 2))
im = np.delete(im, 0, 0)
im = np.delete(im, len(im[1] - 1), 0)
plt.imshow(im, 'gray', aspect='auto') #Figure 2
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
#This code goes through all the files in the 'images' folder and saves the edges to the array 'demImages'

files = []

for filename in os.listdir('images'):
    if filename.endswith('.tif'):
        files.append(os.path.join('images', filename))

demImages = [0*1 for i in range(len(files) - 1)]

for k in range(len(files) - 1):
    channimage = cv2.flip(cv2.bitwise_not(cv2.imread(files[10])), 0) #import and inverts image
    channimageBW = cv2.cvtColor(gwash, cv2.COLOR_BGR2GRAY) #change to grayscale
    channimageBW = np.array(zip(*channimageBW[::-1]))
    channimageBW = cv2.blur(channimageBW,(5,5))
    channimageBW = channimagehBW[50:len(channimageBW) - 1]
    
#     plt.imshow(channimagehBW, 'gray', aspect='auto') #this is matplotlib solution (Figure 1)
#     plt.xticks([]), plt.yticks([])
#     plt.show()


    ret,thresh1 = cv2.threshold(gwashBW, 5, 255,cv2.THRESH_BINARY)

    kernel = np.ones((5,5),np.uint8) #square image kernel used for erosion
    erosion = cv2.erode(thresh1, kernel,iterations = 1) #refines all edges in the binary image

    opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

#     plt.imshow(closing, 'gray', aspect='auto') #Figure 2
#     plt.xticks([]), plt.yticks([])
#     plt.show()

    test2, contours, hierarchy = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)


    areas = [] #list to hold all areas

    for contour in contours:
      ar = cv2.contourArea(contour)
      areas.append(ar)

    sortareas = areas[:]
    sortareas.sort(reverse = True)

    max_areas_index = []

    for i in range(2):
        max_areas_index.append(areas.index(sortareas[i]))
        #index of the list element with largest area

    cnt = [0,0]

    for i in range(2):
        cnt[i] = contours[max_areas_index[i]]

    cnt = np.array(cnt)

    blank = np.array([[0]*len(closing[1]) for i in range(len(closing))])
    im = (cv2.drawContours(blank, cnt, -1, (1, 1, 1), 1)) #(255,255,255)
    im = np.delete(im, 0, 0);
    im = np.delete(im, len(im[1]) - 1, 0);
    im = np.array(im[0:len(im[1])-1])
#     plt.imshow(im, 'gray') #Figure 2
#     plt.show()
#     cv2.imwrite(os.path.join('processed', '%05d.jpg' % (k)),im)
    
    safespace = 50
    detect = 0
    edges = [[0]*5 for i in range(len(im))]
    
    
    for i in range(len(im)):
        ft = np.nonzero(im[i])
        detect = 0
        for j in range(len(ft[0])):
            edges[i][0] = i * scale
            edges[i][1] = ft[0][0] * scale

            if (ft[0][j] - ft[0][j - 1] > safespace):
                detect = detect + 1
                edges[i][detect + 1] = ft[0][j] * scale
     
    demImages[k] = edges

In [ ]:
#This code plots the data from above. 

# axes = plt.gca()
# axes.set_xlim([0,len(demImages[0])])
# axes.set_ylim([0,1200])

for i in range(len(demImages) - 1):
    for j in range(1,len(demImages[i][0])):
        if (i == 0):
            if (j == 1):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], '--', color='blue');
            if (j == 2):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], '--', color='orange');
            if (j == 3):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], '--', color='green');
            if (j == 4):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], '--', color='red');  
                
        if (i > 0):
            if (j == 1):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], alpha=0.08, color='blue');
            if (j == 2):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], alpha=0.08, color='orange');
            if (j == 3):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], alpha=0.08, color='green');
            if (j == 4):
                plt.plot([row[0] for row in demImages[1]], [row[j] for row in demImages[i]], alpha=0.08, color='red');     
            

plt.xlabel('x-Distance ($\mu m$)');
plt.ylabel('y-Distance ($\mu m$)');
plt.title(r'Channel Edges');

plt.savefig('ChannelEdges.png', dpi=600, facecolor='w', edgecolor='w',
        orientation='landscape', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None);

In [ ]:
#This code adjusts for any vertical shift of the sample relative to the microscope objective.

unmodaverage = np.mean(demImages[0][0:10], axis=0)

alignedEdges = [0*1 for i in range(len(demImages) - 1)]

for k in range(len(demImages) - 1):
    tempEdges = [[0]*5 for i in range(len(demImages[k]))]
    unmodaverage = np.mean(demImages[k][0:10], axis=0)
    for i in range(len(demImages[k])):
        tempEdges[i][0] = i * scale
        for j in range(1, len(demImages[k][j])):
            tempEdges[i][j] = demImages[k][i][j] - unmodaverage[1]
   
    alignedEdges[k] = tempEdges

    
for i in range(len(alignedEdges) - 1):
    for j in range(1,len(alignedEdges[i][1])):
        if (i == 0):
            if (j == 1):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], '--', color='blue');
            if (j == 2):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], '--', color='orange');
            if (j == 3):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], '--', color='green');
            if (j == 4):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], '--', color='red');  
                
        if (i > 0):
            if (j == 1):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], alpha=0.08, color='blue');
            if (j == 2):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], alpha=0.08, color='orange');
            if (j == 3):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], alpha=0.08, color='green');
            if (j == 4):
                plt.plot([row[0] for row in alignedEdges[1]], [row[j] for row in alignedEdges[i]], alpha=0.08, color='red'); 

plt.xlabel('x-Distance ($\mu m$)');
plt.ylabel('y-Distance ($\mu m$)');
plt.title(r'Aligned Channel Edges');   

plt.savefig('AlignedEdges.png', dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None);

In [ ]:
#This code allows you to see the shift in each edge relative to its original position

chanaverage = np.mean(edges[0:10], axis=0)

normalEdges = [0*1 for i in range(len(alignedEdges) - 1)]

# axes = plt.gca()
# axes.set_xlim([400,1200])
# axes.set_ylim([-100,100])

for k in range(len(alignedEdges) - 1):
    tempEdges = [[0]*5 for i in range(len(alignedEdges[k]))]
    chanaverage = np.mean(alignedEdges[0][len(alignedEdges[0]) - 50:len(alignedEdges[0]) - 1], axis=0)
    for i in range(len(alignedEdges[k])):
        tempEdges[i][0] = i * scale
        for j in range(1, len(alignedEdges[k][j])):
            tempEdges[i][j] = alignedEdges[k][i][j] - chanaverage[j]
   
    normalEdges[k] = tempEdges

    
for i in range(len(normalEdges) - 1):
    for j in range(1,len(normalEdges[i][1])):
        if (i == 0):
            if (j == 1):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], '--', color='blue');
            if (j == 2):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], '--', color='orange');
            if (j == 3):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], '--', color='green');
            if (j == 4):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], '--', color='red');  
                
        if (i > 0):
            if (j == 1):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], alpha=0.08, color='blue');
            if (j == 2):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], alpha=0.08, color='orange');
            if (j == 3):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], alpha=0.08, color='green');
            if (j == 4):
                plt.plot([row[0] for row in normalEdges[1]], [row[j] for row in normalEdges[i]], alpha=0.08, color='red'); 

# plt.plot([row[0] for row in normalEdges[1]], [row[1] for row in normalEdges[1]]);
# plt.plot([row[0] for row in normalEdges[1]], [row[2] for row in normalEdges[1]]);
# plt.plot([row[0] for row in normalEdges[1]], [row[3] for row in normalEdges[1]]);
# plt.plot([row[0] for row in normalEdges[1]], [row[4] for row in normalEdges[1]]);

plt.xlabel('x-Distance ($\mu m$)');
plt.ylabel('y-Shift ($\mu m$)');
plt.title(r'Normalized Channel Edges');   

plt.savefig('NormalizedEdges.png', dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None);

In [ ]:
#This code graphs the change in average y-position of edges 3 and 4 over time

chanaverage = np.mean(edges[0:10], axis=0)

# axes = plt.gca()
# axes.set_xlim([0,40])
# axes.set_ylim([-50,50])

d4Compare = [[0]*3 for i in range(len(alignedEdges) - 1)]

for k in range(len(alignedEdges) - 1):
    chanaverage1 = np.mean(alignedEdges[0][len(alignedEdges[0]) - 100:len(alignedEdges[0]) - 1], axis=0) 
    chanaveragei = np.mean(alignedEdges[k][len(alignedEdges[0]) - 100:len(alignedEdges[0]) - 1], axis=0) 
    d4Compare[k][0] = k
    for j in range(1, len(alignedEdges[k][j])):
        if (j == 1):
            d4Compare[k][j] = chanaveragei[3] - chanaveragei[2] - (chanaverage1[3] - chanaverage1[2])
        if (j == 2):
            d4Compare[k][j] = chanaveragei[4] - chanaveragei[2] - (chanaverage1[4] - chanaverage1[2])

    
for j in range(3):
    if (j == 1):
        plt.plot([row[0] for row in d4Compare], [row[j] for row in d4Compare], color='green');
    if (j == 2):
        plt.plot([row[0] for row in d4Compare], [row[j] for row in d4Compare], color='red');

#plt.plot([row[0] for row in d4Compare], [row[2] for row in d4Compare], color='red')


plt.xlabel('time (min)');
plt.ylabel('shift wrt edge 2 ($\mu m$)');
plt.title(r'Distance from edge 2');   

plt.savefig('ShiftinTBAFoverTime.png', dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None);